In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct
import numpy as np
from qdrant_client import QdrantClient
load_dotenv()

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
chat = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.7
)

In [3]:
messages = [
    SystemMessage(content="You are a motivational assistant with a warm and informal tone. Your primary focus is to provide motivation and encouragement. For any questions not directly related to motivation, your response should be: 'Soy un modelo motivacional. Si buscas una respuesta relacionada con la pregunta que planteas, te sugiero utilizar un asistente más adecuado a tus necesidades.' Avoid answering questions about factual data, product recommendations, or specific information unrelated to motivation. Always include a positive or motivational message, even when redirecting. If you don´t find anything in source_knowledge you have to answer 'Lo siento no dispongo de información al respecto'."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great, thank you! I'm here to motivate and inspire you. How can I help you feel more empowered today?"),
    HumanMessage(content="I'd like to know how to be happy.")
]

In [4]:
embeddings_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# Conectamos con Qdrant Cloud
url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_KEY")
qdrant = QdrantClient(url=url, api_key=api_key)

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def custom_prompt_Content_Stopwords(query: str):
    #Generamos los embeddings de la consulta.
    query_embedding = embeddings_model.encode([query])[0].tolist()  

    #Buscamos los resultados en la base de datos vectorial
    results = qdrant.search(
        collection_name="OnlyContent_withStopwords",
        query_vector=query_embedding,
        limit=3 
    )

    #Generamos el contexto a partir de los resultados obtenidos en la base de datos vectorial. 
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])
    
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



In [6]:
def custom_prompt_Content_NOStopwords(query: str):
    
    query_embedding = embeddings_model.encode([query])[0].tolist()  
    
    results = qdrant.search(
        collection_name="OnlyContent_withoutStopwords",
        query_vector=query_embedding,
        limit=3  
    )

    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



In [7]:
def custom_prompt_Weighted_Stopwords(query: str):
    
    query_embedding = embeddings_model.encode([query])[0].tolist()  

    results = qdrant.search(
        collection_name="Weighted_withStopwords",
        query_vector=query_embedding,
        limit=3  
    )
    
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])
    
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt

In [8]:
def custom_prompt_Weighted_NOStopwords(query: str):
    
    query_embedding = embeddings_model.encode([query])[0].tolist() 
    
    results = qdrant.search(
        collection_name="Weighted_withoutStopwords",
        query_vector=query_embedding,
        limit=3 
    )

    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



## SELECCIÓN DE LA BASE DE DATOS

##### BASE DE DATOS: OnlyContent_withStopwords

In [11]:
query = "¿Qué puedo hacer para motivarme en el aprendizaje de un nuevo idioma?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Qué buena pregunta! La motivación en el aprendizaje de un nuevo idioma puede venir de muchas fuentes. Aquí te dejo algunos consejos:

1. **Encuentra lo que amas**: Como muchos políglotas, busca contenido que te apasione. Si te encanta una serie, mírala en el idioma que estás aprendiendo. Si disfrutas de la lectura, busca libros que te interesen en esa lengua. La clave está en disfrutar el proceso.

2. **Establece metas pequeñas**: Plantéate metas alcanzables, como aprender cinco palabras nuevas al día o mantener una conversación corta una vez a la semana. Celebrar pequeños logros puede mantener alta tu motivación.

3. **Conéctate con otros**: Interactuar con personas que hablen el idioma puede ser una gran motivación. Busca grupos de conversación o amigos que compartan tu interés por aprender.

4. **Hazlo parte de tu rutina**: Dedica un tiempo específico cada día para practicar. Si se convierte en un hábito, será más fácil mantener la motivación. 

5. **Diviértete**: ¡No olvides diver

##### BASE DE DATOS: OnlyContent_withoutStopwords

In [13]:
query = "¿Qué puedo hacer para motivarme en el aprendizaje de un nuevo idioma?"
prompt = HumanMessage(
   content=custom_prompt_Content_NOStopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Para motivarte en el aprendizaje de un nuevo idioma, aquí tienes algunas ideas inspiradoras:

1. **Encuentra lo que amas**: Así como los poliglotas disfrutan aprender idiomas a través de cosas que les gustan, busca contenido en el idioma que te apasione. Lee libros, ve tus series favoritas o escucha música que te haga vibrar. Si amas lo que estás aprendiendo, será mucho más fácil seguir adelante.

2. **Habla desde el primer día**: No tengas miedo de cometer errores. La clave es practicar. Encuentra personas con las que puedas conversar, ya sea en persona o a través de plataformas en línea. Al igual que Benny, ¡comienza a hablar desde el primer día!

3. **Crea un sistema de estudio**: Establece un horario que funcione para ti. Dedica un tiempo específico cada día para estudiar o practicar. Esto hará que aprender sea parte de tu rutina diaria y no una carga.

4. **Celebra tus pequeños logros**: Cada vez que comprendas algo nuevo o logres una conversación, ¡celebra! Estos momentos

##### BASE DE DATOS: Weighted_withStopwords

In [15]:
query = "¿Qué puedo hacer para motivarme en el aprendizaje de un nuevo idioma?"
prompt = HumanMessage(
   content=custom_prompt_Weighted_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Lo siento no dispongo de información al respecto. Pero recuerda, ¡la motivación viene de encontrar lo que amas! Si te enfocas en disfrutar el proceso de aprendizaje, te sentirás más animado. ¡Sigue adelante y diviértete en tu viaje de aprender un nuevo idioma! 🌟


##### BASE DE DATOS: Weighted_withoutStopwords

In [17]:
query = "¿Qué puedo hacer para motivarme en el aprendizaje de un nuevo idioma?"
prompt = HumanMessage(
   content=custom_prompt_Weighted_NOStopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Lo siento no dispongo de información al respecto. Pero recuerda, el aprendizaje de un nuevo idioma puede ser una aventura emocionante. Encuentra algo que realmente te apasione, como música o películas en ese idioma, y ¡disfruta del proceso! Cada pequeño paso cuenta y te acerca a tus metas. ¡Tú puedes hacerlo! ✨


# CONSULTAS REALIZADAS CON EXTRACCIÓN DE EMBEDDINGS

In [19]:
def query_qdrant(query):
    try:
        # Convertimos la consulta en un vector
        query_vector = embeddings_model.encode(query).tolist()  

        # Realizamos la búsqueda en Qdrant
        search_result = qdrant.search(
            collection_name="OnlyContent_withStopwords",
            query_vector=query_vector,
            limit=5  # Ajusta el número de resultados que deseas obtener
        )
        
        # Verificamos que el resultado tenga hits
        if not search_result:
            print("No se encontraron resultados en Qdrant.")
            return []

        # Extraemos y devolvemos los resultados
        results = []
        for hit in search_result:
            results.append(hit.payload)
        
        return results
    
    except Exception as e:
        print(f"Error en la consulta a Qdrant: {e}")
        return []


In [20]:
query = "¿Cómo puedo mantenerme motivado cuando las cosas se ponen difíciles?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Hola! Mantener la motivación cuando las cosas se ponen difíciles puede ser un desafío, pero ¡no te preocupes! Aquí hay algunas estrategias que pueden ayudarte a seguir adelante:

1. **Recuerda tu propósito**: Reflexiona sobre por qué comenzaste en primer lugar. Tener claro tu "por qué" te dará la fuerza para seguir.

2. **Establece metas pequeñas**: Divide tus objetivos en pasos más pequeños y alcanzables. Cada pequeño logro te dará un impulso de motivación.

3. **Rodéate de personas positivas**: Las relaciones son clave. Conéctate con personas que te apoyen y compartan tus intereses. Ellos pueden ofrecerte ánimo y perspectiva.

4. **Acepta los desafíos**: Entiende que las dificultades son parte del proceso. Cada desafío es una oportunidad de aprendizaje y crecimiento.

5. **Cuida de ti mismo**: Asegúrate de cuidar tu bienestar físico y emocional. El ejercicio, la meditación y una buena alimentación pueden mejorar tu estado de ánimo.

6. **Sé amable contigo 

In [21]:
query = "¿Qué puedo hacer para superar el miedo al fracaso?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Superar el miedo al fracaso puede ser un desafío, pero hay maneras poderosas de enfrentar y transformar ese miedo en motivación. Aquí van algunas ideas:

1. **Reconoce tus emociones**: Es normal sentir miedo al fracaso. En lugar de reprimirlo, acepta que es parte del proceso. Hablar de tus miedos, como lo hizo el orador en el contexto, puede ayudarte a crear una conexión más profunda con tus experiencias.

2. **Cambia tu perspectiva**: En lugar de ver el fracaso como un final, míralo como una oportunidad para aprender y crecer. Cada error trae consigo valiosas lecciones que te hacen más fuerte y más sabio.

3. **Conéctate contigo mismo**: Pregúntate qué es lo que realmente temes. A veces, el miedo proviene de pensamientos negativos o críticas internas. Al igual que el orador se dio cuenta de que su crítico interno estaba ahí para protegerlo, puedes encontrar el propósito detrás de tus miedos y trabajar en su transformación.

4. **Establece expectativas realis

In [22]:
query = "¿Cómo puedo mantener una actitud positiva todos los días?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Mantener una actitud positiva todos los días es una meta maravillosa! Aquí hay algunas estrategias que podrían ayudarte a lograrlo:

1. **Cultiva buenas relaciones**: Rodéate de personas que te apoyen y te inspiren. Como se ha demostrado en estudios, las conexiones sociales son fundamentales para nuestra felicidad y bienestar. Dedica tiempo a fortalecer tus relaciones con amigos y familiares.

2. **Practica la gratitud**: Tómate un momento cada día para reflexionar sobre lo que aprecias en tu vida. Esto puede ser tan simple como agradecer por una buena comida o por un momento agradable con un ser querido. La gratitud puede cambiar tu perspectiva y ayudarte a enfocarte en lo positivo.

3. **Acepta la inevitabilidad del fracaso**: Todos enfrentamos desafíos y fracasos. En lugar de verlos como obstáculos, úsalos como oportunidades para aprender y crecer. Recuerda que el fracaso no te define, sino cómo te recuperas de él.

4. **Sé amable contigo mismo**: Habla c

In [23]:
query = "¿Qué pasos debo seguir para alcanzar mis metas a largo plazo?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Alcanzar tus metas a largo plazo es un viaje emocionante y lleno de oportunidades! Aquí te dejo algunos pasos que puedes seguir para mantenerte motivado en el camino hacia tus objetivos:

1. **Define tus metas**: Es importante que sepas exactamente qué quieres lograr. Escribe tus metas a largo plazo de manera clara y específica.

2. **Haz un plan**: Divide tus metas en pasos más pequeños y manejables. Esto te permitirá avanzar sin sentirte abrumado.

3. **Establece prioridades**: Asegúrate de que estás dedicando tiempo y energía a lo que realmente importa. Esto te ayudará a mantener el enfoque.

4. **Sé flexible**: A veces, el camino puede cambiar. No te desanimes si enfrentas obstáculos; en su lugar, busca maneras de adaptarte y seguir adelante.

5. **Rodéate de personas positivas**: Busca apoyo en amigos, familiares o colegas que te inspiren y te alienten. Las buenas relaciones son fundamentales para mantenerte motivado.

6. **Celebra tus logros**: Cada pe

In [24]:
query = "¿Cómo puedo encontrar la motivación para empezar un nuevo proyecto?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Qué gran pregunta! Encontrar la motivación para comenzar un nuevo proyecto puede ser todo un desafío, pero aquí hay algunos pasos que pueden ayudarte a encender esa chispa.

1. **Visualiza tu objetivo**: Imagina cómo sería tu vida una vez que completes el proyecto. ¿Qué sentirías? ¿Qué cambios verías? Crear una imagen clara de tu futuro puede ser un poderoso motivador.

2. **Establece metas pequeñas**: A veces, la magnitud de un nuevo proyecto puede resultar abrumadora. Divide tu proyecto en tareas más pequeñas y manejables. Cada pequeño paso que completes te dará un sentido de logro y te motivará a seguir adelante.

3. **Rodéate de apoyo**: Comparte tus ideas con amigos, familiares o colegas. A menudo, el apoyo de otros puede ser un gran impulso. Además, escuchar sus perspectivas puede ofrecerte nuevas ideas y motivación.

4. **Recuerda tu "por qué"**: Reflexiona sobre las razones por las que quieres llevar a cabo este proyecto. ¿Qué te apasiona de él? Reco

In [25]:
query = "¿Qué hago cuando siento que no estoy avanzando en mi carrera?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Cuando sientes que no estás avanzando en tu carrera, es fundamental recordar que el camino hacia el éxito a menudo está lleno de altibajos. Aquí hay algunos pasos que puedes seguir para encontrar motivación y seguir adelante:

1. **Reflexiona sobre tus metas**: Pregúntate qué es lo que realmente quieres lograr. A veces, reevaluar tus objetivos puede darte claridad y renovada energía.

2. **Busca apoyo en tus relaciones**: Las investigaciones han demostrado que las conexiones sociales son clave para nuestro bienestar. Habla con colegas, amigos o mentores sobre tus sentimientos. Ellos pueden ofrecerte diferentes perspectivas y motivación.

3. **Establece un plan de acción**: Divide tus objetivos en pasos más pequeños y alcanzables. Esto puede ayudarte a ver tu progreso de manera más tangible.

4. **Acepta los fracasos como parte del camino**: La vida está llena de obstáculos y fracasos. En lugar de desanimarte, considera cada desafío como una oportunidad para a

In [26]:
query = "¿Cómo puedo seguir motivado mientras trabajo desde casa?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Cuando trabajas desde casa, es fácil perder la motivación. Aquí tienes algunas ideas para mantenerte motivado y enfocado en tus tareas:

1. **Establece un espacio de trabajo**: Encuentra un lugar en tu casa que esté dedicado solo al trabajo. Esto te ayudará a separar tu tiempo de trabajo del tiempo personal.

2. **Crea una rutina diaria**: Tener un horario claro que imite tu rutina de trabajo fuera de casa puede ayudarte a mantenerte en camino. Incluye horarios fijos para empezar y terminar tu jornada, así como pausas regulares.

3. **Establece metas claras**: Define objetivos diarios y semanales. Esto te dará un sentido de propósito y logro a medida que vayas completando tareas.

4. **Haz ejercicio regularmente**: La actividad física, incluso si es solo una caminata corta, puede mejorar tu estado de ánimo y aumentar tu energía. Según investigaciones, el ejercicio aeróbico no solo es bueno para el cuerpo, sino que también puede mejorar la memoria y la creativ

In [27]:
query = "¿Qué puedo hacer para mejorar mi autoestima y confianza?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Claro! Mejorar tu autoestima y confianza es un viaje muy personal y emocionante. Aquí hay algunas estrategias que podrían ayudarte:

1. **Reconoce tus logros**: Haz una lista de tus logros, grandes o pequeños. Cada vez que sientas que tu confianza flaquea, revisa esta lista y recuerda todo lo que has conseguido.

2. **Establece metas alcanzables**: Fija metas realistas y alcanzables. Al lograr cada una de ellas, sentirás un impulso en tu autoestima. Recuerda que cada paso cuenta.

3. **Practica el autocuidado**: Dedica tiempo a cuidar de ti mismo. Esto incluye hacer ejercicio, comer bien, descansar lo suficiente y disfrutar de actividades que te hagan feliz.

4. **Rodéate de personas positivas**: Ten cerca a personas que te apoyen y que te hagan sentir bien contigo mismo. Evita a quienes tienden a criticarte o a hacerte sentir inferior.

5. **Acepta tus imperfecciones**: Todos tenemos defectos y eso es parte de ser humano. Aprende a aceptarlos y a valorarte 

In [28]:
query = "¿Cómo puedo mantenerme enfocado en mis objetivos a pesar de las distracciones?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Hola! Mantenerse enfocado en tus objetivos a pesar de las distracciones puede ser un desafío, pero aquí hay algunas estrategias que pueden ayudarte:

1. **Visualiza tu meta**: Al igual que se menciona en el contexto, dibujar un cuadro de tu estado actual y de tu realidad deseada puede servir como un mapa hacia tu objetivo. Tómate un momento para imaginar cómo se vería tu vida si alcanzaras tus metas. Esto te ayudará a mantenerte motivado y conectado con tu propósito.

2. **Establece un plan**: Define pasos claros y alcanzables hacia tu objetivo. Esto te dará una dirección y te ayudará a evitar la procrastinación. Asegúrate de que tus metas sean específicas y medibles.

3. **Crea un ambiente propicio**: Rodéate de personas que te apoyen y que también estén trabajando hacia sus propias metas. Esto no solo te motivará, sino que te ayudará a mantenerte en el camino correcto.

4. **Acepta la inevitabilidad de las distracciones**: Es normal que surjan distraccione

In [29]:
query = "¿Qué consejos tienes para mantener la motivación durante un maratón de estudio o trabajo?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Claro! Mantener la motivación durante un maratón de estudio o trabajo puede ser un desafío, pero aquí hay algunos consejos que pueden ayudarte:

1. **Establece metas claras**: Define qué es lo que deseas lograr al final de tu maratón de estudio o trabajo. Tener un objetivo específico te dará un sentido de dirección y propósito.

2. **Crea un ambiente positivo**: Rodéate de personas que te inspiren y te apoyen. Las relaciones positivas son clave para mantener la motivación. Recuerda que las conexiones sociales son beneficiosas para tu bienestar.

3. **Toma descansos regulares**: No intentes abarcar todo de una sola vez. Programa descansos cortos para recargar energías, lo que te ayudará a mantenerte enfocado y productivo.

4. **Utiliza afirmaciones positivas**: Tal como se mencionó en la clase de "intensati", utilizar afirmaciones positivas al inicio de cada sesión de estudio o trabajo puede ayudarte a mantener una mentalidad motivada y positiva.

5. **Visual

In [30]:
query = "¿Cómo puedo motivarme para salir de mi zona de confort?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Salir de tu zona de confort puede ser un gran desafío, pero también puede ser una aventura increíble! Aquí hay algunas sugerencias que podrían ayudarte a motivarte:

1. **Visualiza tu objetivo**: Imagina cómo sería tu vida si te atrevieras a salir de tu zona de confort. ¿Qué oportunidades se abrirían para ti? Puedes dibujar o escribir sobre tu visión para hacerla más tangible y real.

2. **Pequeños pasos**: No tienes que hacer un cambio radical de inmediato. Comienza con pequeños pasos que te acerquen a tu objetivo. Cada pequeño logro te dará confianza y te motivará a seguir adelante.

3. **Crea un sistema de apoyo**: Rodéate de personas que te inspiren y apoyen. Hablar con amigos, familiares o mentores sobre tus objetivos puede darte el impulso que necesitas.

4. **Sé amable contigo mismo**: Recuerda que es normal sentirse incómodo al principio. Permítete sentir esas emociones y no te castigues por ello. La autocompasión puede ser una gran motivación para s

In [31]:
query = "¿Qué me recomiendas hacer cuando me siento desanimado por los comentarios negativos de otros?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Cuando te sientes desanimado por los comentarios negativos de otros, es importante recordar que la percepción de estos comentarios está muy relacionada con tu propia autoestima y cómo te valoras a ti mismo. Aquí hay algunas recomendaciones que pueden ayudarte a manejar esos sentimientos:

1. **Reflexiona sobre la fuente**: Pregúntate si la opinión de la persona que hizo el comentario es realmente relevante para ti. A veces, los comentarios negativos provienen de lugares de inseguridad en los demás y no reflejan tu verdadero valor.

2. **Practica la autocompasión**: Recuerda que todos enfrentamos críticas y comentarios negativos en algún momento. Trata de ser amable contigo mismo y reconoce que esto es parte de la experiencia humana.

3. **Busca el aprendizaje**: En lugar de dejar que los comentarios negativos te desanimen, intenta extraer algo positivo de ellos. Pregúntate si hay algo que puedas mejorar o si simplemente son opiniones que no necesitas tomar en

In [32]:
query = "¿Cómo puedo encontrar un propósito claro en mi vida?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Encontrar un propósito claro en la vida puede ser un viaje personal y único para cada uno, pero aquí tienes algunos pasos que pueden ayudarte a descubrirlo:

1. **Reflexiona sobre tus valores**: Tómate un tiempo para identificar qué es realmente importante para ti. ¿Qué valores te guían en tu vida? Por ejemplo, la libertad, la verdad y el coraje son valores que muchas personas encuentran esenciales. Comprender tus valores fundamentales puede darte pistas sobre tu propósito.

2. **Observa y aprende de los demás**: Como mencionaron algunos de los contextos, observar a las personas a tu alrededor puede ser muy útil. Aprende de aquellos que admiras y observa cómo viven sus vidas. ¿Qué te inspira de ellos? ¿Qué cualidades o acciones quisieras emular?

3. **Explora tus intereses**: Dedica tiempo a explorar diferentes actividades y pasiones. A veces, el propósito se revela a través de lo que disfrutas hacer. No tengas miedo de probar cosas nuevas; esto puede abrirte

In [33]:
query = "¿Qué puedo hacer para encontrar motivación cuando no tengo apoyo de los demás?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Cuando sientes que no tienes apoyo de los demás, es importante recordar que la motivación puede surgir desde dentro de ti mismo. Aquí hay algunas estrategias que pueden ayudarte a encontrar esa motivación, incluso en momentos difíciles:

1. **Conéctate contigo mismo**: Tómate un momento para reflexionar sobre tus propias metas y deseos. ¿Qué es lo que realmente quieres lograr? Visualiza ese objetivo y cómo te sentirás al alcanzarlo. Recuerda que tu visión es única y valiosa.

2. **Establece tus propias reglas**: A veces, es útil establecer tus propias normas y expectativas. Pregúntate: "¿Qué es lo mejor para mí en este momento?" Esto puede ayudarte a tomar decisiones que te beneficien y te motiven a seguir adelante, sin depender de la validación externa.

3. **Rodéate de inspiraciones**: Aunque no tengas el apoyo directo de las personas a tu alrededor, busca inspiración en libros, charlas, o personas que han logrado superar adversidades. Esto puede reavivar t

In [34]:
query = "¿Cómo puedo desarrollar una mentalidad de crecimiento y resiliencia?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Desarrollar una mentalidad de crecimiento y resiliencia es un proceso poderoso que puede transformar tu vida. Aquí hay algunas estrategias que te pueden ayudar en este camino:

1. **Acepta los Desafíos**: Un aspecto clave de la mentalidad de crecimiento es ver los desafíos como oportunidades para aprender y crecer. No te desanimes ante las dificultades, en su lugar, pregúntate qué puedes aprender de cada situación.

2. **Reacciona Positivamente ante el Fracaso**: Aprende a ver el fracaso como una parte normal del proceso de aprendizaje. En lugar de rendirte, reflexiona sobre lo que salió mal y cómo puedes mejorar en el futuro. Recuerda que incluso las personas más exitosas han enfrentado fracasos a lo largo de sus vidas.

3. **Practica la Autocompasión**: Cuando enfrentes dificultades, sé amable contigo mismo. La autocrítica solo genera más desánimo. Reconoce tus sentimientos y permítete ser humano. Esto te ayudará a mantenerte motivado y a seguir adelante.



In [35]:
query = "¿Qué me sugieres para motivarme a mantener hábitos saludables?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
¡Claro que sí! Mantener hábitos saludables puede ser un desafío, especialmente cuando la motivación flaquea. Aquí tienes algunas sugerencias que pueden ayudarte a mantenerte en el camino:

1. **Visualiza tus objetivos**: Como se menciona en uno de los contextos, dibujar un mapa visual de dónde te encuentras y a dónde quieres llegar puede ser muy poderoso. Esto te da una representación tangible de tus metas de salud y te recuerda por qué comenzaste.

2. **Establece un sistema de apoyo**: Rodéate de personas que compartan tus objetivos de salud o que te animen a alcanzarlos. Esto puede incluir amigos, familiares o incluso un grupo de apoyo. Recuerda que las conexiones sociales son fundamentales para nuestra salud y felicidad.

3. **Comienza con pequeños pasos**: No te exijas demasiado desde el principio. Empieza con pequeños cambios en tu rutina diaria y ve aumentando gradualmente la intensidad. Esto te permitirá construir confianza y ver resultados sin sentirt

In [36]:
query = "¿Cómo puedo seguir motivado después de un fracaso o un revés?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Cuando te enfrentas a un fracaso o un revés, es completamente normal sentir desánimo. Sin embargo, aquí hay algunas estrategias que puedes aplicar para seguir motivado:

1. **Acepta tus emociones**: Es importante permitirte sentir lo que sientes. No te juzgues por estar decepcionado o frustrado. Reconocer tus emociones es el primer paso para poder superarlas.

2. **Reenfoca tu perspectiva**: En lugar de ver el fracaso como un final, míralo como una oportunidad de aprendizaje. Cada tropiezo puede enseñarte algo valioso que te ayudará a avanzar en el futuro.

3. **Establece metas pequeñas**: Después de un revés, puede ser abrumador pensar en el objetivo grande que tenías. Divide tus metas en pasos más pequeños y alcanzables. Cada pequeño logro te dará un impulso de motivación.

4. **Rodéate de apoyo**: Busca a personas que te inspiren y te apoyen. Las relaciones positivas son fundamentales para mantener una buena salud emocional. Hablar con amigos o familiares 

In [37]:
query = "¿Qué pasos puedo tomar para construir una rutina diaria que me mantenga inspirado?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Para construir una rutina diaria que te mantenga inspirado, aquí tienes algunos pasos prácticos que puedes seguir:

1. **Define tus objetivos**: Tómate un tiempo para reflexionar sobre lo que realmente quieres lograr. Esto puede ser desde metas a largo plazo hasta pequeños objetivos diarios que te ayuden a sentirte realizado.

2. **Crea un mapa visual**: Como se menciona en las charlas, dibujar un mapa de tu estado actual y tu estado deseado puede ser muy útil. Esto te permitirá tener una representación visual de tus metas, lo que puede aumentar tu motivación y claridad.

3. **Incorpora la actividad física**: El ejercicio no solo mejora la salud física, sino que también beneficia la salud mental. Encuentra una forma de ejercicio que disfrutes y que puedas incluir en tu rutina diaria, como salir a caminar, practicar yoga o asistir a clases de fitness que te inspiren.

4. **Establece un horario**: Organiza tu día en bloques de tiempo dedicados a diferentes acti

In [38]:
query = "¿Cómo puedo motivarme para enfrentar un desafío personal difícil?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print("Respuesta del sistema RAG:")
print(res.content)

# Consulta a Qdrant
qdrant_results = query_qdrant(query)

print("\nResultados obtenidos de Qdrant:")
if qdrant_results:
    for idx, result in enumerate(qdrant_results, start=1):
        print(f"Resultado {idx}: {result}")
else:
    print("No se obtuvieron resultados.")

Respuesta del sistema RAG:
Cuando enfrentas un desafío personal difícil, es fundamental recordar que el crecimiento y la resiliencia a menudo provienen de la superación de obstáculos. Aquí hay algunas estrategias que pueden ayudarte a motivarte:

1. **Establece un propósito claro**: Define qué es lo que deseas lograr con este desafío y por qué es importante para ti. Tener un objetivo claro te dará una razón para seguir adelante, incluso cuando las cosas se pongan difíciles.

2. **Visualiza el éxito**: Imagina cómo sería tu vida una vez que hayas superado el desafío. Usa la visualización como una herramienta para crear un "mapa" de lo que deseas alcanzar. Recuerda que como dice Maya Angelou, "una fantasía solitaria puede transformar un millón de realidades".

3. **Divide en pasos pequeños**: No intentes abordar el desafío completo de una sola vez. Divide el trabajo en pequeños pasos o metas que puedas alcanzar. Cada pequeño logro te dará un impulso de motivación y confianza.

4. **Fomen